# Congestion Charges - Hard

You may need to create views to complete these questions - but you do not have permission to create tables or views in the default schema. Your SQL commands are executed by user scott in schema gisq - you may create or drop views and tables in schema scott but not in gisq.

In [1]:
# Prerequesites
from pyhive import hive
%load_ext sql
%sql hive://cloudera@quickstart.cloudera:10000/sqlzoo
%config SqlMagic.displaylimit = 20

## 1.
When creating a view in scott you must specify the schema name of the sources and the destination.

In [2]:
%%sql
CREATE SCHEMA IF NOT EXISTS scott

 * hive://cloudera@quickstart.cloudera:10000/sqlzoo
Done.


[]

## 2.
There are four types of permit. The most popular type means that this type has been issued the highest number of times. Find out the most popular type, together with the total number of permits issued.

In [3]:
%%sql
SELECT chargetype, COUNT(*) n
  FROM permit
    GROUP BY chargetype
    ORDER BY n DESC
    LIMIT 1

 * hive://cloudera@quickstart.cloudera:10000/sqlzoo
Done.


chargetype,n
Daily,27


## 3.
For each of the vehicles caught by camera 19 - show the registration, the earliest time at camera 19 and the time and camera at which it left the zone.

In [4]:
%%sql
WITH t AS (
  -- registrations showing at camera 19, the earliest time
  SELECT reg, MIN(whn) earliest
    FROM image JOIN vehicle ON (image.reg=vehicle.id)
      WHERE image.camera=19
      GROUP BY reg
)
SELECT a.reg, a.earliest, a.nextone, image.camera FROM
 (SELECT t.reg, t.earliest, MIN(image.whn) AS nextone
  FROM t LEFT JOIN image ON (
      t.reg=image.reg)
  WHERE t.earliest<image.whn
  GROUP BY t.reg, t.earliest) a JOIN image ON (
    a.reg=image.reg AND a.nextone=image.whn)

 * hive://cloudera@quickstart.cloudera:10000/sqlzoo
Done.


reg,earliest,nextone,camera
SO 02 CSP,2007-02-25 07:51:10.0,2007-02-25 07:55:11.0,18
SO 02 DSP,2007-02-25 16:31:01.0,2007-02-25 17:42:41.0,19
SO 02 JSP,2007-02-25 17:14:11.0,2007-02-25 17:17:03.0,3
SO 02 TSP,2007-02-25 07:23:00.0,2007-02-25 07:26:31.0,19


## 4.
For all 19 cameras - show the position as IN, OUT or INTERNAL and the busiest hour for that camera.

In [5]:
%%sql
WITH t AS (
  SELECT camera, COALESCE(camera.perim, 'INTERNAL') perim,
    HOUR(image.whn) hr, COUNT(*) n
      FROM camera JOIN image ON (camera.id=image.camera)
      GROUP BY camera, camera.perim, HOUR(image.whn)
)
SELECT camera, perim, hr, MAX(n) max_n
  FROM t
    GROUP BY camera, perim, hr

 * hive://cloudera@quickstart.cloudera:10000/sqlzoo
Done.


camera,perim,hr,max_n
1,IN,6,1
2,IN,7,1
3,IN,17,3
3,IN,18,2
5,IN,7,1
8,IN,7,2
9,OUT,6,1
9,OUT,16,6
9,OUT,18,1
10,OUT,5,1


## 5.
Anomalous daily permits. Daily permits should not be issued for non-charging days. Find a way to represent charging days. Identify the anomalous daily permits.

In [6]:
%%sql
SELECT * FROM permit
  WHERE DATE_FORMAT(sdate, 'u') IN (6, 7)
    AND chargetype='Daily'

 * hive://cloudera@quickstart.cloudera:10000/sqlzoo
Done.


reg,sdate,chargetype
SO 02 ATP,2007-01-21 00:00:00.0,Daily
SO 02 BTP,2007-02-03 00:00:00.0,Daily
SO 02 BTP,2007-02-04 00:00:00.0,Daily
SO 02 CTP,2007-01-21 00:00:00.0,Daily
SO 02 FTP,2007-02-25 00:00:00.0,Daily
SO 02 HTP,2006-01-21 00:00:00.0,Daily
SO 02 HTP,2006-01-22 00:00:00.0,Daily
SO 02 JTP,2007-01-21 00:00:00.0,Daily


## 6.
Issuing fines: Vehicles using the zone during the charge period, on charging days must be issued with fine notices unless they have a permit covering that day. List the name and address of such culprits, give the camera and the date and time of the first offence.

In [8]:
%%sql
WITH t AS (
-- vehicles with permits
  SELECT permit.reg, sdate, keeper,
    CASE WHEN chargetype='Daily' THEN DATE_ADD(sdate, 1)
        WHEN chargetype='Weekly' THEN DATE_ADD(sdate, 7)
        WHEN chargetype='Monthly' THEN ADD_MONTHS(sdate, 1)
        WHEN chargetype='Annual' THEN ADD_MONTHS(sdate, 12)
        END AS edate
      FROM vehicle LEFT JOIN
      permit ON (vehicle.id=permit.reg)
), f AS (
  SELECT image.reg, name, address, camera, whn
    FROM t RIGHT JOIN image ON (t.reg=image.reg) JOIN
      keeper ON (keeper.id=t.keeper)
      WHERE DATE_FORMAT(whn, 'u') BETWEEN 1 AND 5 AND
        NOT image.whn BETWEEN t.sdate AND t.edate
)
SELECT a.reg, a.name, a.address, a.first_offence, b.camera FROM
 (SELECT reg, name, address, MIN(whn) first_offence FROM f 
  GROUP BY reg, name, address) a JOIN
    (SELECT * FROM f) b ON (a.reg=b.reg AND a.first_offence=b.whn)

 * hive://cloudera@quickstart.cloudera:10000/sqlzoo
Done.


reg,name,address,first_offence,camera
